In [1]:
import pandas as pd
import numpy as np
import requests as rq
import json
import imdb
import requests
from bs4 import BeautifulSoup
import re

In [2]:
osc_bp = pd.read_csv('../data/scraping_results/osc_bp.csv')
print('Liczba filmów nominowanych do Oscara od 1927 roku: ', len(osc_bp))
osc_bp

Liczba filmów nominowanych do Oscara od 1927 roku:  591


,year,film,wiki,winner
0,1927,Wings,/wiki/Wings_(1927_film),True
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False
2,1927,The Racket,/wiki/The_Racket_(1928_film),False
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True
4,1928,Alibi,/wiki/Alibi_(1929_film),False
...,...,...,...,...
586,2022,The Fabelmans,/wiki/The_Fabelmans,False
587,2022,Tár,/wiki/T%C3%A1r,False
588,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False
589,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False


In [3]:
osc_bp.tail(10)

,year,film,wiki,winner
581,2022,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once,True
582,2022,All Quiet on the Western Front,/wiki/All_Quiet_on_the_Western_Front_(2022_film),False
583,2022,Avatar: The Way of Water,/wiki/Avatar:_The_Way_of_Water,False
584,2022,The Banshees of Inisherin,/wiki/The_Banshees_of_Inisherin,False
585,2022,Elvis,/wiki/Elvis_(2022_film),False
586,2022,The Fabelmans,/wiki/The_Fabelmans,False
587,2022,Tár,/wiki/T%C3%A1r,False
588,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False
589,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False
590,2022,Women Talking,/wiki/Women_Talking_(film),False


funkcja która pobiera info o genres z mojo

In [7]:


def scrape_genres(movie_title):
    # Build the search URL
    search_url = f'https://www.boxofficemojo.com/search/?q={movie_title}'

    # Send the search request
    search_response = requests.get(search_url)
    search_soup = BeautifulSoup(search_response.text, 'html.parser')

    # Extract the link to the movie's box office page
    movie_links = search_soup.find_all('a', class_='a-link-normal')
    if len(movie_links) == 0:
        return None
    movie_link = None
    for link in movie_links:
        if '/title/' in link['href']:
            movie_link = link
            break
    if movie_link is None:
        return None
    movie_url = f'https://www.boxofficemojo.com{movie_link["href"]}'

    # Send the box office request
    box_office_response = requests.get(movie_url)
    box_office_soup = BeautifulSoup(box_office_response.text, 'html.parser')

    genres = []
    try:
        genre_div = box_office_soup.find('div', class_='mojo-summary-values')
        if genre_div:
            genres_element = genre_div.find('span', string='Genres')
            if genres_element:
                genres = genres_element.find_next_sibling('span').text.strip()
    except (AttributeError, TypeError):
        pass
    if genres:
        genres= re.sub(r'\n\s+', ',', genres).strip(', ')
        genres= re.sub(r'\s*,\s*', ',', genres).lower()
    print(genres, movie_title)

    return genres


In [12]:
genres_df = osc_bp.copy()

In [ ]:

# Create an empty DataFrame to store the scraped results
scraped_results = pd.DataFrame(columns=['film', 'genres'])

# Iterate over each row in the DataFrame
for index, row in genres_df.iterrows():
    film_title = row['film']
    genres = scrape_genres(film_title)
    
    # Add the scraped results to the DataFrame
    scraped_results = scraped_results.append({'film': film_title, 'genres': genres}, ignore_index=True)
    
    # Save the results to a CSV file
    scraped_results.to_csv('../data/additional/genres_scraped.csv', index=False)



In [26]:
genres_scraped= pd.read_csv('../data/additional/genres_scraped.csv')
genres_scraped=genres_scraped.drop('Unnamed: 0', axis=1)
genres_scraped

,film,genres
0,Wings,"action,drama,romance,war"
1,7th Heaven,drama
2,The Racket,"drama,fantasy,mystery,sci-fi,thriller"
3,The Broadway Melody,"Drama,Musical,Romance"
4,In Old Arizona,"Short,Western"
...,...,...
569,The Fabelmans,"Comedy,Talk-Show"
570,The Banshees of Inisherin,"Comedy,Drama"
571,Triangle of Sadness,"Comedy,Drama"
572,Avatar: The Way of Water,"Action,Adventure,Fantasy"


In [20]:
genres_scraped = pd.read_csv('../data/additional/genres_scraped.csv').drop('Unnamed: 0', axis=1)
# Convert values in 'Genre' column to lowercase
genres_scraped['genres'] = genres_scraped['genres'].str.lower()
genres_scraped.tail(10)


,film,genres
564,All Quiet on the Western Front,"drama,war"
565,Elvis,"biography,drama,music"
566,Everything Everywhere All at Once,"action,adventure,comedy"
567,Tár,"drama,music"
568,Top Gun: Maverick,"comedy,talk-show"
569,The Fabelmans,"comedy,talk-show"
570,The Banshees of Inisherin,"comedy,drama"
571,Triangle of Sadness,"comedy,drama"
572,Avatar: The Way of Water,"action,adventure,fantasy"
573,Women Talking,drama


In [17]:
genres_scraped.to_csv('../data/additional/genres_scraped.csv')

In [38]:
#filmy pojawiają się wielokrotnie z różną ilością genres - zostawiam tylko ten wiersz w którym genres jest najwięcej
genres_scraped['genre_count'] = genres_scraped['genres'].str.count(',') + 1
genres_scraped = genres_scraped.sort_values('genre_count', ascending=False)
genres_scraped.drop('genre_count', axis=1, inplace=True)
genres_scraped.dropna(inplace=True)
print('znaleziono genres dla', len(genres_scraped), 'filmów z', len(genres_df))

znaleziono genres dla 574 filmów z 591


dodaje kolumnę genres do genres_not_presentdf będącej kopią osc_bp

In [41]:
# Merge the two dataframes based on the "film" column
genres_added = genres_df.merge(genres_scraped[['film', 'genres']], on='film', how='left')

# Drop rows with NaN values in the "genres" column
genres_added = genres_added.dropna(subset=['genres']).drop_duplicates(subset='wiki', keep='first').reset_index().drop(['index', 'winner'], axis=1)

genres_added.to_csv('../data/additional/genres_final.csv', index=False)
# Display the updated dataframe
genres_added.tail(10)


,year,film,wiki,genres
564,2022,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once,"action,adventure,comedy"
565,2022,All Quiet on the Western Front,/wiki/All_Quiet_on_the_Western_Front_(2022_film),"drama,war"
566,2022,Avatar: The Way of Water,/wiki/Avatar:_The_Way_of_Water,"action,adventure,fantasy"
567,2022,The Banshees of Inisherin,/wiki/The_Banshees_of_Inisherin,"comedy,drama"
568,2022,Elvis,/wiki/Elvis_(2022_film),"biography,drama,music"
569,2022,The Fabelmans,/wiki/The_Fabelmans,"comedy,talk-show"
570,2022,Tár,/wiki/T%C3%A1r,"drama,music"
571,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,"comedy,talk-show"
572,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,"comedy,drama"
573,2022,Women Talking,/wiki/Women_Talking_(film),drama


RATING

In [39]:
import imdb

def get_imdb_scores(movie_title):
    """
    Given a movie title, returns a dictionary containing the IMDb score and number of votes for the movie.
    """
    ia = imdb.IMDb()
    scores = {}
    # search for the movie by title
    results = ia.search_movie(movie_title)
    if results:
        # use the first search result to retrieve the IMDb score and number of votes
        movie = results[0]
        ia.update(movie)
        score = movie.get('rating')
        votes = movie.get('votes')
        scores['score'] = score
        scores['votes'] = votes
    else:
        # if no search results are found, add entries with None values
        scores['score'] = None
        scores['votes'] = None
    return scores


In [28]:
rating_df = osc_bp.copy()

In [ ]:

scraped_results_votes = pd.DataFrame(columns=['film', 'rating', 'numVotes'])
# Iterate over each row in the DataFrame
for index, row in rating_df.iterrows():
    film_title = row['film']
    rating = get_imdb_scores(film_title)['score']
    numVotes = get_imdb_scores(film_title)['votes']
    
    # Add the scraped results to the DataFrame
    scraped_results_votes  = scraped_results_votes.append({'film': film_title, 'rating': rating, 'numVotes':numVotes}, ignore_index=True)
    
    # Save the results to a CSV file
    scraped_results_votes.to_csv('../data/additional/ratings_scraped.csv', index=False)




In [42]:

ratings_scraped = pd.read_csv('../data/additional/ratings_scraped.csv').drop('Unnamed: 0', axis=1)
rating_added = rating_df.merge(ratings_scraped[['film', 'rating', 'numVotes']], on='film', how='left')
# Drop rows with NaN values in the "ratings" column
rating_added = rating_added.dropna(subset=['rating']).drop_duplicates('wiki')
print('znaleziono rating dla ', len(rating_added), ' z ', len(osc_bp), 'filmów')
rating_added.to_csv('../data/additional/ratings_final.csv')


znaleziono rating dla  583  z  591 filmów


In [ ]:
rating_db = pd.read_csv('../data/additional/ratings_final.csv')

now adding box office

funkcja która pobiera z mojo info o box office

In [38]:
import requests
from bs4 import BeautifulSoup

def scrape_box_office_info(movie_title):
    # Build the search URL
    search_url = f'https://www.boxofficemojo.com/search/?q={movie_title}'
    
    # Send the search request
    search_response = requests.get(search_url)
    search_soup = BeautifulSoup(search_response.text, 'html.parser')
    
    # Extract the link to the movie's box office page
    movie_links = search_soup.find_all('a', class_='a-link-normal')
    if len(movie_links) == 0:
        print(f'No box office information found for {movie_title}')
        return None
    movie_link = None
    for link in movie_links:
        if '/title/' in link['href']:
            movie_link = link
            break
    if movie_link is None:
        print(f'No box office information found for {movie_title}')
        return None
    movie_url = f'https://www.boxofficemojo.com{movie_link["href"]}'
    # Send the box office request
    box_office_response = requests.get(movie_url)
    box_office_soup = BeautifulSoup(box_office_response.text, 'html.parser')
   

    box_office = {'Domestic': 'N/A', 'International': 'N/A', 'Worldwide': 'N/A'}
    try:
        table = box_office_soup.find('div', class_='a-section a-spacing-none mojo-performance-summary-table')
        rows = table.find_all('div', class_='a-section a-spacing-none')
        for row in rows:
            span = row.find('span', class_='a-size-small')
            if span:
                title = span.text.strip()
                money = row.find('span', class_='money')
                if money:
                    box_office[title.split()[0]] = money.text.strip()
                else:
                    box_office[title.split()[0]] = 'N/A'
    except (AttributeError, TypeError):
        pass
    print(box_office, movie_title)
    
    return box_office

In [ ]:
bo_df = osc_bp.copy()
# Apply the function to each row in the "film" column
bo_df['box_office'] = bo_df['film'].apply(scrape_box_office_info)



In [99]:
#Usunięcie NaN 
bo_df = bo_df.dropna(subset=['box_office'])
# Sprawdzenie liczby duplikatów w kolumnie 'film'
num_duplicates = bo_df['film'].duplicated().sum()
bo_df = bo_df.drop_duplicates(subset=['film'])
# Wyświetlenie liczby duplikatów
print("Usunięto: ", num_duplicates, ' duplikatów')

In [112]:
bo_df.to_csv('../data/additional/bo_scraped.csv')

In [32]:
bo_df = pd.read_csv('../data/additional/bo_scraped.csv')
bo_df = bo_df.drop('Unnamed: 0', axis=1)
bo_df.tail(10)

,year,film,wiki,winner,box_office
549,2022,All Quiet on the Western Front,/wiki/All_Quiet_on_the_Western_Front_(2022_film),False,"{'Domestic': 'N/A', 'International': 'N/A', 'W..."
550,2022,Avatar: The Way of Water,/wiki/Avatar:_The_Way_of_Water,False,"{'Domestic': '$684,016,647', 'International': ..."
551,2022,The Banshees of Inisherin,/wiki/The_Banshees_of_Inisherin,False,"{'Domestic': '$10,582,266', 'International': '..."
552,2022,Elvis,/wiki/Elvis_(2022_film),False,"{'Domestic': '$151,040,048', 'International': ..."
553,2022,The Fabelmans,/wiki/The_Fabelmans,False,"{'Domestic': '$17,348,945', 'International': '..."
554,2022,Tár,/wiki/T%C3%A1r,False,"{'Domestic': '$6,773,650', 'International': '$..."
555,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False,"{'Domestic': '$718,732,821', 'International': ..."
556,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,"{'Domestic': '$4,608,096', 'International': '$..."
557,2022,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once,True,"{'Domestic': '$77,191,785', 'International': '..."
558,2022,Women Talking,/wiki/Women_Talking_(film),False,"{'Domestic': '$5,456,531', 'International': '$..."


In [34]:
import ast

# Definicja funkcji sprawdzającej strukturę wartości box_office
def check_structure(value):
    expected_structure = {'Domestic': None, 'International': None, 'Worldwide': None}
    return isinstance(value, dict) and value.keys() == expected_structure.keys()

# Convert string values to dictionaries
def parse_box_office(value):
    if isinstance(value, str):
        try:
            return ast.literal_eval(value.replace("'", "\""))
        except (ValueError, SyntaxError):
            return None
    return value

bo_cleaned = bo_df.copy()

bo_cleaned['box_office'] = bo_cleaned['box_office'].apply(parse_box_office)


# Wyświetlenie wierszy, w których struktura wartości box_office jest inna niż oczekiwana
mismatched_rows = bo_cleaned[~bo_cleaned['box_office'].apply(check_structure)]

# Wyświetlenie wyniku
print('obecnie ', len(mismatched_rows), 'posiada niepoprawną strukturę danych')


obecnie  0 posiada niepoprawną strukturę danych


In [35]:
bo_final=bo_cleaned.copy()
# Create separate columns for each box office category
bo_final[['domestic_box_office', 'international_box_office', 'worldwide_box_office']] = bo_final['box_office'].apply(pd.Series)
bo_final = bo_final.drop(['box_office', 'domestic_box_office', 'international_box_office'], axis=1)
bo_final = bo_final.dropna(subset=['worldwide_box_office'])
print('znaleziono rating dla ', len(bo_final), ' z ', len(osc_bp), 'filmów')
bo_final.tail(10)

znaleziono rating dla  559  z  591 filmów


,year,film,wiki,winner,worldwide_box_office
549,2022,All Quiet on the Western Front,/wiki/All_Quiet_on_the_Western_Front_(2022_film),False,N/A
550,2022,Avatar: The Way of Water,/wiki/Avatar:_The_Way_of_Water,False,"$2,319,766,200"
551,2022,The Banshees of Inisherin,/wiki/The_Banshees_of_Inisherin,False,"$49,209,277"
552,2022,Elvis,/wiki/Elvis_(2022_film),False,"$288,670,284"
553,2022,The Fabelmans,/wiki/The_Fabelmans,False,"$45,164,110"
554,2022,Tár,/wiki/T%C3%A1r,False,"$27,541,681"
555,2022,Top Gun: Maverick,/wiki/Top_Gun:_Maverick,False,"$1,493,491,858"
556,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,"$25,615,870"
557,2022,Everything Everywhere All at Once,/wiki/Everything_Everywhere_All_at_Once,True,"$140,211,159"
558,2022,Women Talking,/wiki/Women_Talking_(film),False,"$8,954,708"


In [36]:
bo_final.to_csv('../data/additional/bo_final.csv')